In [ ]:
from utils import *

In [3]:
gpt4 = OpenAI_LLM("gpt-4o")

In [7]:
def generate_with_gpt4_cot(api, language):
    with open("prompts/gpt4_cot/plan/instruction.txt", "r") as f:
        user_prompt = f.read().format(api=api, language=language)

    plan = gpt4.free_output(user_prompt)[0]

    with open("prompts/gpt4_cot/execution/instruction.txt", "r") as f:
        user_prompt = f.read().format(api=api, language=language, plan=plan)

    return gpt4.free_output(user_prompt)

In [10]:
result = generate_with_gpt4_cot("gather", "tensorflow")

In [12]:
api_to_post = read_json("api_to_post")
benchmark = read_json("benchmark")

In [13]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

gpt4_cot_result = {}
max_workers = 20


def call_and_wrap(i):
    name = i["name"]
    lang = i["language"]
    result = generate_with_gpt4_cot(name, lang)
    return name, result[0]


with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(call_and_wrap, i) for i in benchmark]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
        name, result = future.result()
        gpt4_cot_result[name] = result

Processing: 100%|██████████| 48/48 [01:30<00:00,  1.89s/it]


In [14]:
output_folder = "gpt4_cot_result"
os.makedirs(output_folder, exist_ok=True)

for api in gpt4_cot_result:
    file_path = os.path.join(output_folder, f"{api}.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(gpt4_cot_result[api])
        print(f"Sentences for API '{api}' written to {file_path}.")

Sentences for API 'Attributes2Impl' written to gpt4_cot_result\Attributes2Impl.txt.
Sentences for API 'AccessibilityRequestPreparer' written to gpt4_cot_result\AccessibilityRequestPreparer.txt.
Sentences for API 'FragmentTransaction' written to gpt4_cot_result\FragmentTransaction.txt.
Sentences for API 'URISyntaxException' written to gpt4_cot_result\URISyntaxException.txt.
Sentences for API 'X509TrustManager' written to gpt4_cot_result\X509TrustManager.txt.
Sentences for API 'ConnectionEvent' written to gpt4_cot_result\ConnectionEvent.txt.
Sentences for API 'CheckBoxPreference' written to gpt4_cot_result\CheckBoxPreference.txt.
Sentences for API 'MediaRouter' written to gpt4_cot_result\MediaRouter.txt.
Sentences for API 'LinkedList' written to gpt4_cot_result\LinkedList.txt.
Sentences for API 'NotificationManager' written to gpt4_cot_result\NotificationManager.txt.
Sentences for API 'FragmentManager' written to gpt4_cot_result\FragmentManager.txt.
Sentences for API 'Attr' written to gp